In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, BatchNormalization, Flatten, InputLayer, LeakyReLU, Conv2DTranspose, Reshape  
from tensorflow.keras import Model

import matplotlib.pyplot as plt

import os
import time

#### Dataset Description

##### Human Faces Dataset (128x128x3)

This dataset is a comprehensive collection of over 50,000 images, exclusively focusing on human faces. The images in this dataset are of size 128 pixels in width and 128 pixels in height. Each image represents a human face captured from various individuals, encompassing diverse expressions, poses, and ethnicities. However, the dataset contains several inherent challenges:

1. *Off-center faces*: Not all faces in the images are perfectly centered. Some images may have faces positioned towards the edges or corners.

2. *Multiple faces*: Occasionally, an image may contain multiple faces. This includes scenarios where two or more individuals are captured in a single image.

3. *Occluding objects*: In some images, objects or obstructions may partially or completely occlude the faces. These occluding objects can include accessories like hats, scarves, or sunglasses, as well as other objects that unintentionally obstruct the facial features.

4. *Overexposure*: Some images in the dataset may suffer from overexposure due to excessive lighting conditions. These overexposed images can affect the visibility and quality of facial features, posing a challenge for face recognition and analysis algorithms.

5. *Non-uniform backgrounds*: The backgrounds of the images are not consistent across the dataset. The variations in backgrounds include different colors, textures, and environmental settings. This diversity in backgrounds introduces additional complexity for tasks such as face segmentation and background removal.

##### Dataset Contents

The dataset includes:

- Over 50,000 high-resolution images of human faces.
- Images are stored in a standardized format of 128x128 pixels.
- Each image is meticulously curated and annotated for accurate analysis.
   
![Portion of the dataset](./data/illustration/dataset_illustration.png)

#### Data Preprocessing

Data preprocessing is an essential step in preparing data for analysis or machine learning tasks. It involves transforming and cleaning the raw data to make it suitable for further processing. Let's explore the specific steps involved in this process:

##### 1. Data Acquisition

The first step is to read the 50,000 images. This may involve loading the images from a local directory, a remote server, or a dataset repository. The goal is to have the raw data ready for further processing. In our case, we're reading a local directory.

##### 2. Data Normalization

To ensure consistency and improved model convergence, it's important to normalize the values of the images. In our case, we're normalizing the pixel values between -1 and 1.

##### 3. Shuffle and Batch Creation

To introduce randomness and prevent any potential biases in the data ordering, we can shuffle the images before creating batches. This ensures that the order of the images does not influence the learning process. After shuffling, we can proceed to create batches of 32 images.

##### 4. Batch Processing

To efficiently process large datasets, it's common to divide them into smaller batches. In this case, we have created batches of 32 images. This allows us to feed the data in smaller portions during training or analysis, reducing memory requirements and enabling potentially enabling parallel processing if available.

By dividing the data into batches, we can iteratively process each batch without loading the entire dataset into memory at once. This approach is particularly useful when working with large datasets that do not fit entirely into memory.

These steps ensure that the data is properly prepared for subsequent analysis or machine learning tasks. It's important to note that these steps can be customized based on specific requirements and the nature of the dataset.


In [ ]:
def load_and_preprocess_dataset(directory, image_size, batch_size, shuffle=True):
    """
    function that reads the Dataset, Shuffle it, normalizes it and batches it 
    """
    dataset = tf.keras.utils.image_dataset_from_directory(
        directory=directory,
        image_size=image_size,
        batch_size=batch_size,
        shuffle=shuffle,
        label_mode=None, 
        color_mode="rgb"  # RGB Color
    )

    # # Normalizing Dataset between -1 and 1
    dataset = dataset.map(lambda x: x / 255.0 * 2.0 - 1.0)

    return dataset

TRAIN_DIR = "./data/data/train/real"
IMAGE_SIZE = (128,128)
BATCH_SIZE = 32

# Chargement des données d'entraînement
train_data = load_and_preprocess_dataset(TRAIN_DIR, IMAGE_SIZE, BATCH_SIZE, shuffle=True)

#### Discriminator

Next step is to create The discriminator. The discriminator is a key component of a Generative Adversarial Network (GAN). Its role is to distinguish between real and generated (fake) samples. Here's a brief description of the discriminator:

- Input: The discriminator takes as input images or samples from the generator. In the case of image generation tasks, the input typically consists of images with 128*128*3 dimensions.

- Architecture: The discriminator usually consists of layers that process the input and extract features. Common architectural choices include convolutional layers, followed by activation functions such as ReLU or LeakyReLU. These layers are designed to capture relevant patterns and discriminative information from the input samples.

- Output: The output of the discriminator is a probability score that represents the likelihood of the input being real or fake. It is usually a single scalar value between 0 and 1. A value closer to 1 indicates that the input is classified as real, while a value closer to 0 suggests that it is classified as fake.

The discriminator's objective is to improve its ability to differentiate between real and generated samples, which in turn drives the generator to produce more realistic outputs. The discriminator and generator are trained in an adversarial manner, where the generator aims to fool the discriminator, while the discriminator aims to accurately classify the samples.

In [ ]:
class Discriminator (Model):
    def __init__(self, input_shape, batch_size):
        super(Discriminator, self).__init__()

        self.__input_shape = input_shape
        self.__batch_size = batch_size

        self.__discriminator = tf.keras.Sequential(
            [
                InputLayer(input_shape=self.__input_shape, batch_size=self.__batch_size),
                Conv2D(64,(3,3),padding="same", strides=2),
                BatchNormalization(),
                Conv2D(64,(3,3),padding="same", strides=2),
                BatchNormalization(),
                LeakyReLU(alpha=0.01),
                Conv2D(128,(3,3),padding="same", strides=2),
                BatchNormalization(),
                LeakyReLU(alpha=0.01),
                Conv2D(256,(3,3),padding="same", strides=2),
                BatchNormalization(),
                LeakyReLU(alpha=0.01),
                Flatten(),
                Dense (1, activation ="sigmoid")
            ]
        )

    def call (self, input) :
        return self.__discriminator(input)

    def get_model (self) :
        return self.__discriminator

#### Generator

The generator is a fundamental component of a Generative Adversarial Network (GAN). Its purpose is to generate synthetic samples that resemble real data. Let's take a quick look at the generator:

- Input: The generator typically takes random noise or a latent vector as input. This vector is often sampled from a probability distribution, such as a uniform or Gaussian distribution. The size and dimensionality of the input vector depend on the specific problem and the desired output.

- Architecture: The generator consists of layers that transform the input noise or latent vector into meaningful data representations. Common choices include fully connected (dense) layers or transposed convolutional layers. These layers gradually upsample the input and apply non-linearities to generate higher-resolution outputs.

- Output: The output of the generator is a synthetic sample that aims to resemble the real data. In image generation tasks, the output can be an image with specific dimensions, such as 32x32 or 64x64 pixels. The generator generates samples that lie within the same data distribution as the real samples.

- Training: The generator is trained in conjunction with the discriminator. Its objective is to produce synthetic samples that can fool the discriminator into classifying them as real. The generator's parameters are updated through backpropagation and optimization techniques, such as stochastic gradient descent (SGD) or Adam, based on the feedback from the discriminator.

The generator's goal is to progressively improve its ability to generate realistic and diverse samples. It learns to capture the underlying patterns and structures present in the real data, effectively synthesizing new samples from random noise. As the training progresses, the generator becomes more proficient in generating samples that resemble the true data distribution.

In summary, the generator in a GAN framework acts as a creative component, producing synthetic samples that progressively become more indistinguishable from real data. It plays a crucial role in challenging the discriminator and enhancing the overall performance of the GAN model.


In [ ]:
class Generator (Model):
    def __init__(self, batch_size):
        super(Generator, self).__init__()

        self.__generator = tf.keras.Sequential(
            [
                InputLayer(input_shape=(64,), batch_size=batch_size),
                Dense (8*8*64, use_bias=False),
                LeakyReLU(alpha=0.01),
                Reshape ((8,8,64)),
                Conv2DTranspose(filters=256, padding="same", kernel_size=(3,3), strides = 2),
                BatchNormalization (),
                LeakyReLU(alpha=0.01),
                Conv2DTranspose(filters=256, padding="same", kernel_size=(3,3), strides = 2),
                BatchNormalization (),
                LeakyReLU(alpha=0.01),
                Conv2DTranspose(filters=256, padding="same", kernel_size=(3,3), strides = 2),
                BatchNormalization (),
                LeakyReLU(alpha=0.01),
                Conv2DTranspose(filters=128, padding="same", kernel_size=(3,3), strides = 2),
                BatchNormalization (),
                LeakyReLU(alpha=0.01),
                Conv2D(3,(3,3),padding="same")
            ]
        )

    def call (self, input) :
        return self.__generator(input)

    def get_model (self) :
        return self.__generator

In [ ]:
dis = Discriminator((128,128,3),BATCH_SIZE)
gen = Generator(BATCH_SIZE)

###  Training 

- Define the loss functions: Specify the loss functions for the generator and discriminator. The discriminator's loss aims to correctly classify real and fake samples, while the generator's loss encourages the generated samples to be classified as real.

In [18]:
cross_entropy = tf.keras.losses.BinaryCrossentropy()


- Discriminator loss

This function calculates the loss for the discriminator during training. It takes two inputs: `real_output` and `fake_output`, which represent the discriminator's predictions for real and fake samples, respectively.

The function uses the `cross_entropy` loss function to compare the discriminator's predictions to the target labels. For real samples, the target label is 1, indicating that the sample is real. For fake samples, the target label is 0, indicating that the sample is fake.

The `real_loss` is computed by comparing the real output predictions to a tensor of ones with the same shape as `real_output`. This loss represents how well the discriminator can correctly classify real samples.

Similarly, the `fake_loss` is computed by comparing the fake output predictions to a tensor of zeros with the same shape as `fake_output`. This loss represents how well the discriminator can correctly classify fake samples.

Finally, the `total_loss` is calculated by summing the `real_loss` and `fake_loss`. This combined loss is used to optimize the discriminator's weights during training.

The function then returns the `total_loss`, which represents the overall loss of the discriminator.

In [20]:
def discriminator_loss(real_output, fake_output):
    
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    
    return total_loss


- Generator loss

This function calculates the loss for the generator in a Generative Adversarial Network (GAN) during training. It takes `fake_output` as input, which represents the discriminator's predictions for the generated (fake) samples.

The function uses the `cross_entropy` loss function to compare the discriminator's predictions for the fake samples to a tensor of ones with the same shape as `fake_output`. The target label is 1, indicating that the desired outcome for the generator is to fool the discriminator into classifying the fake samples as real.

The `gen_loss` is computed by comparing the fake output predictions to the tensor of ones. This loss represents how well the generator is performing in generating samples that can deceive the discriminator.

Finally, the function returns the `gen_loss` as a float value.

In [22]:
def generator_loss(fake_output):
    
    gen_loss = cross_entropy(tf.ones_like(fake_output), fake_output)

    return float (gen_loss)

In [19]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
checkpoint_dir = './training_checkpoints_V3'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=gen,
                                 discriminator=dis)

ckpt_manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir, max_to_keep=3)
if ckpt_manager.latest_checkpoint:
    checkpoint.restore(ckpt_manager.latest_checkpoint)

In [ ]:
EPOCHS = 50
noise_dim = 64
num_examples_to_generate = 16

# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = gen(noise, training=True)

      real_output = dis(images, training=True)
      fake_output = dis(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, gen.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, dis.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, gen.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, dis.trainable_variables))

In [ ]:
def gen_and_show_image (model, epoch, noise) :
    gen_image = model(noise, training=False)
    plt.imshow(gen_image[0])
    plt.axis('off')
    plt.savefig('./images/image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

In [ ]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_step(image_batch)
      # Generate after the final epoch
    
    gen_and_show_image(
        gen,
        epoch,
        seed
      )

    checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

In [ ]:
train(train_data, EPOCHS)

#### Result

![Training evolution](./data/illustration/training.gif)
